In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import glob
import math
import time
import torch
import numpy as np
from itertools import product
from collections import defaultdict
from collections import OrderedDict as odict
from netCDF4 import Dataset
from os.path import exists
import os
from textwrap import dedent
from matplotlib.ticker import EngFormatter
import seaborn as sns

# Instructions for Creating Histograms on a New Dataset

To add a new dataset, follow these steps:

1. Define the appropriate set of configurations for the dataset in the `info_fam` dictionary below. 
    
    Let's assume your new data family's name is `"01_cares"`.

2. Convert this notebook to a python script and run it like below:

    ```bash
    source ./activate mamba
    jupytext --to py notebooks/n02_histdata.ipynb
    NWORKERS=20
    for MYRANK in $(seq 0 $((NWORKERS-1))); do 
        python notebooks/n02_histdata.py -f "01_cares" -r ${MYRANK} -s ${NWORKERS} &
    done
    wait
    ```

    Remember to specify a new `"01_cares"` dataset name and number of workers `NWORKERS`. 

    This step will generate `NWORKERS` files such as `f'{data_dir}'/02_masshist/04_caressamp_00.nc` to `f'{data_dir}'/02_masshist/04_caressamp_19.nc`.
    `

3. Come back to this notebook, and go all the way down to the "Compiling All Data into a Single netCDF File" section:

    * Replace the `fam_data` from `None` to the new data family name `"01_cares"`.
    
    * Run the cell script to generate the compiled `f'{data_dir}'/02_masshist/04_caressamp.nc` file.

In [ ]:
# Note: The following should be specified properly.
data_dir = '../data'
datalfs_dir = '/path/to/large/file/system/data_partnn'

out_datadir, out_datalfsdir = data_dir, datalfs_dir

args_datafamily, args_dryrun, args_belazy = '01_bwchi', False, True

my_rank, n_workers = None, 1

use_tqdm, tqdm = False, lambda itrtr: itrtr
if use_tqdm and (my_rank == 0): 
    from tqdm import tqdm

In [ ]:
def get_histdata(nc_fp, nc_path, snridx_nc, n_bins, diam_low, diam_high, diam_logbase, 
    chem_species_keep, lib='numpy', verbose=False):

    if snridx_nc in (None, 'all'):
        i_setstrt, i_setend = 0, None
    else:
        # The start indices for each dataset of particles. 
        # This is one-based (i.e., fortran-style indices).
        i_setstrts = nc_fp.variables['part_start_index'][:] - 1

        n_dsets = len(i_setstrts)
        assert isinstance(snridx_nc, int), f'"{snridx_nc}" is not an integer index'

        snridxpos_nc = snridx_nc + n_dsets if snridx_nc < 0 else snridx_nc
        assert (snridxpos_nc < n_dsets) and (snridxpos_nc >= 0), dedent(f'''
            There are only "{n_dsets}" start indices, and you're 
            trying to access the "{snridx_nc}"-th set of particles:
                nc_path = {nc_path}
                snridx_nc = {snridx_nc}
                n_dsets = {n_dsets}''')
        
        i_setstrt = i_setstrts[snridxpos_nc]
        if snridxpos_nc == (n_dsets - 1):
            i_setend = None 
        else:
            i_setend = i_setstrts[snridxpos_nc + 1]

    # Getting the number of chemicals and the actual
    n_chem_og, = nc_fp.variables['aero_species'].shape
    chem_species_ogstr = nc_fp.variables['aero_species'].names
    assert isinstance(chem_species_ogstr, str)

    chem_species_og = chem_species_ogstr.split(',')
    assert len(chem_species_og) == n_chem_og

    # Keeping the specified subset of chemical species
    assert isinstance(chem_species_keep, str)
    if chem_species_keep == 'all':
        # The number of kept chemical species
        n_chem = n_chem_og
        # The kept chemical species names list
        chem_species = chem_species_og
        # The kept chemical subset indices
        i_chemkeep = slice(None, None, None)
    else:
        # The kept chemical species names list
        chem_species = chem_species_keep.split(',')
        # The number of kept chemical species
        n_chem = len(chem_species)
        # Sanity checks
        for chm in chem_species:
            assert chm in chem_species_og, dedent(f'''
                The "{chm}" chemical specie could not be found in the input 
                file's chemical species:
                    available chemical species: "{chem_species_ogstr}"''')
        # The kept chemical subset indices
        i_chemkeep = [chem_species_og.index(chm) for chm in chem_species]
    
    chem_species_str = ','.join(chem_species)
    
    # The 2-d mass array of the chemicals in particles
    m_chmprt = nc_fp.variables['aero_particle_mass'][i_chemkeep, i_setstrt:i_setend]
    n_part, = m_chmprt.shape[1:]
    assert m_chmprt.shape == (n_chem, n_part)
    assert m_chmprt.dtype == np.float64, m_chmprt.dtype

    # The chemical density for each species in (kg/m^3)
    rho_chem = nc_fp.variables['aero_density'][i_chemkeep].astype(np.float64)
    assert rho_chem.shape == (n_chem,)
    assert rho_chem.dtype == np.float64, rho_chem.dtype

    # The importance weight of each particle
    c_part = nc_fp.variables['aero_num_conc'][i_setstrt:i_setend]
    assert c_part.shape == (n_part,)
    assert c_part.dtype == np.float64, c_part.dtype

    if lib == 'torch':
        m_chmprt = torch.tensor(m_chmprt)
        rho_chem = torch.tensor(rho_chem)
        c_part = torch.tensor(c_part)
    elif lib == 'numpy':
        pass
    else:
        raise ValueError(f'lib={lib} is not implemented')

    # The volume of each chemical species within particles
    v_chempart = m_chmprt / rho_chem.reshape(n_chem, 1)
    assert v_chempart.shape == (n_chem, n_part)

    # The volume of each particle
    v_part = v_chempart.sum(axis=0)
    assert v_part.shape == (n_part,)

    # The diameter of each particle
    d_part = (6 * v_part / np.pi) ** (1 / 3)
    assert d_part.shape == (n_part,)

    # The mass of each particle
    m_part = m_chmprt.sum(axis=0)
    assert m_part.shape == (n_part,)

    # The normalized weight of chemicals within each particle
    w_chmprt = m_chmprt / m_part.reshape(1, n_part)
    assert w_chmprt.shape == (n_chem, n_part)

    # The volume of each chemical species within particles
    v_chempart = m_chmprt / rho_chem.reshape(n_chem, 1)
    assert v_chempart.shape == (n_chem, n_part)

    # The volume of each particle
    v_part = v_chempart.sum(axis=0)
    assert v_part.shape == (n_part,)

    # The diameter of each particle
    d_part = (6 * v_part / np.pi) ** (1 / 3)
    assert d_part.shape == (n_part,)

    # The mass of each particle
    m_part = m_chmprt.sum(axis=0)
    assert m_part.shape == (n_part,)

    # The normalized weight of chemicals within each particle
    w_chmprt = m_chmprt / m_part.reshape(1, n_part)
    assert w_chmprt.shape == (n_chem, n_part)

    # The log-dimeter logistics
    logdiam_low, logdiam_high = np.log(diam_low) / diam_logbase, np.log(diam_high) / diam_logbase
    if verbose and not (d_part >= diam_low).all():
        print(f'\n{nc_path} has diameters as small as {d_part.min()}.')
    if verbose and not (d_part < diam_high).all():
        print(f'\n{nc_path} has diameters as large as {d_part.max()}.')

    # The log-diameter of all particles
    logd_part = np.log(d_part) / diam_logbase
    assert logd_part.shape == (n_part,)

    # The normalized log-dimater within the `[logdiam_high, logdiam_low]` range.
    logdn_part = (logd_part - logdiam_low) / (logdiam_high - logdiam_low)
    assert logdn_part.shape == (n_part,)

    # The log-dimater bin of each particle
    dbin_part = (logdn_part * n_bins)
    dbin_part = (dbin_part.astype(np.int64) if lib == 'numpy'
        else dbin_part.to(torch.long))
    dbin_part[dbin_part >= n_bins] = n_bins - 1
    dbin_part[dbin_part < 0] = 0
    assert dbin_part.shape == (n_part,)

    if lib == 'numpy':
        # The number of particles histogram within each diamater bin
        n_prthst = np.zeros(n_bins, np.int64)
        np.add.at(n_prthst, dbin_part, c_part)
        assert n_prthst.shape == (n_bins,)

        # The total mass of particles within each diameter bin
        m_prthst = np.zeros(n_bins, np.float64)
        np.add.at(m_prthst, dbin_part, m_part * c_part)
        assert m_prthst.shape == (n_bins,)

        # The total mass of each chemical species in each diameter bin
        m_chmprthst_ = np.zeros((n_bins, n_chem), np.float64)
        np.add.at(m_chmprthst_, dbin_part, m_chmprt.T * c_part.reshape(n_part, 1))
        assert m_chmprthst_.shape == (n_bins, n_chem)
    elif lib == 'torch':
        # The number of particles histogram within each diamater bin
        n_prthst = torch.zeros(n_bins, dtype=c_part.dtype)
        n_prthst.scatter_add_(dim=0, index=dbin_part, src=c_part)
        assert n_prthst.shape == (n_bins,)

        # The total mass of particles within each diameter bin
        m_prthst = torch.zeros(n_bins, dtype=m_part.dtype)
        m_prthst.scatter_add_(dim=0, index=dbin_part, src=m_part * c_part)
        assert m_prthst.shape == (n_bins,)

        # The total mass of each chemical species in each diameter bin
        m_chmprthst_ = torch.zeros(n_bins, n_chem, dtype=m_part.dtype)
        m_chmprthst_.scatter_add_(dim=0, index=dbin_part.reshape(n_part, 1), 
            src=m_chmprt.T * c_part.reshape(n_part, 1))
        assert m_chmprthst_.shape == (n_bins, n_chem)
    else:
        raise ValueError(f'lib={lib} is not implemented')

    # Making the mass data "channels"-compatible
    m_chmprthst = m_chmprthst_.T
    assert m_chmprthst.shape == (n_chem, n_bins)

    if lib == 'torch':
        m_chmprt = m_chmprt.detach().cpu().numpy()
        rho_chem = rho_chem.detach().cpu().numpy()
        c_part = c_part.detach().cpu().numpy()
    elif lib == 'numpy':
        pass
    else:
        raise ValueError(f'lib={lib} is not implemented')

    assert n_prthst.dtype == np.int64, n_prthst.dtype
    assert m_prthst.dtype == np.float64, m_prthst.dtype
    assert m_chmprthst.dtype == np.float64, m_chmprthst.dtype

    out_dict = dict(n_prthst=n_prthst, m_prthst=m_prthst, 
        m_chmprthst=m_chmprthst, chem_species_str=chem_species_str)

    return out_dict

def get_bwncpath(data_dir, data_tree, i_grid, n_grid, scnr_idx, t_idx):
    n_x, n_y, n_z = n_grid
    i_x, i_y, i_z = i_grid
    assert scnr_idx == i_z + i_y * n_z + i_x * n_y * n_z

    nc_path = f'{data_dir}/{data_tree}/scenario_{scnr_idx}/scenario_{scnr_idx}_0001_000000{t_idx+1:02d}.nc'
    snridx_nc = 'all'
    assert exists(nc_path), dedent(f'''
        The BlueWater chi-sampling netCDF data file is missing:
            nc_path = {nc_path}''')

    return nc_path, snridx_nc

def get_caresncpath(data_dir, data_tree, i_grid, n_grid, scnr_idx, t_idx):
    n_x, n_y, n_z = n_grid
    i_x, i_y, i_z = i_grid
    assert scnr_idx == i_z + i_y * n_z + i_x * n_y * n_z

    nc_path = f'{data_dir}/{data_tree}/{t_idx+1:02d}/cares_{i_x+1:03d}_{i_y+1:03d}_{t_idx+1:08d}.nc'
    snridx_nc = i_z

    if (t_idx != 3):
        is_ncokay = True
    elif (i_x < 94):
        is_ncokay = True
    elif (i_x == 94) and (i_y < 13):
        is_ncokay = True
    else:
        is_ncokay = False
    
    if is_ncokay:
        assert exists(nc_path), dedent(f'''
            The CARES simulation netCDF data file is missing:
                nc_path = {nc_path}''')
    else:
        nc_path, snridx_nc = get_caresncpath(data_dir, data_tree, i_grid, n_grid, scnr_idx, t_idx + 1)

    return nc_path, snridx_nc

# The chemical densities in the scenarios library
chem2rho_bwchi = {'SO4': 1800.0, 'NO3': 1800.0, 'Cl': 2200.0, 'NH4': 1800.0, 'MSA': 1800.0, 
    'ARO1': 1400.0, 'ARO2': 1400.0, 'ALK1': 1400.0, 'OLE1': 1400.0, 'API1': 1400.0, 'API2': 1400.0, 
    'LIM1': 1400.0, 'LIM2': 1400.0, 'CO3': 2600.0, 'Na': 2200.0, 'Ca': 2600.0, 'OIN': 2600.0, 
    'OC': 1000.0, 'BC': 1800.0, 'MOC': 1000.0, 'H2O': 1000.0}

# The chemical kappa values in the scenarios library
chem2kappa_bwchi = {'SO4': 0.65, 'NO3': 0.65, 'Cl': 0.53, 'NH4': 0.65, 'MSA': 0.53, 
    'ARO1': 0.1, 'ARO2': 0.1, 'ALK1': 0.1, 'OLE1': 0.1, 'API1': 0.1, 'API2': 0.1, 
    'LIM1': 0.1, 'LIM2': 0.1, 'CO3': 0.53, 'Na': 0.53, 'Ca': 0.53, 'OIN': 0.001, 
    'OC': 0.001, 'BC': 0.0, 'MOC': 0.001, 'H2O': 0.0}

# The optical wave-lengths and water's refraction index
wvid2len = {f'{wvln * 1e9:g}nm': wvln for wvln in [300e-9, 400e-9, 550e-9, 600e-9, 1e-6]}
h2o_refrs = {'300nm/H2O': (1.35+1.524e-08j), '400nm/H2O': (1.34+2.494e-09j), 
    '550nm/H2O': (1.33+0j), '600nm/H2O': (1.33+1.638e-09j), '1000nm/H2O': (1.33+3.128e-06j)}

# The chemical refractive index values in the scenarios library
chem2refr_bwchi1 = {'SO4': (1.5+0j), 'NO3': (1.5+0j), 'Cl': (1.5+0j), 'NH4': (1.5+0j), 
    'MSA': (1.43+0j), 'ARO1': (1.45+0j), 'ARO2': (1.45+0j), 'ALK1': (1.45+0j), 'OLE1': (1.45+0j), 
    'API1': (1.45+0j), 'API2': (1.45+0j), 'LIM1': (1.45+0j), 'LIM2': (1.45+0j), 'CO3': (1.5+0j), 
    'Na': (1.5+0j), 'Ca': (1.5+0j), 'OIN': (1.55+0.006j), 'OC': (1.45+0j), 'BC': (1.82+0.74j), 
    'MOC': (1.5+0j), 'H2O': None}
chem2refr_bwchi = {f'{wvid}/{chem}': chmrfr for wvid in wvid2len 
    for chem, chmrfr in chem2refr_bwchi1.items()}
chem2refr_bwchi.update(h2o_refrs)

# The chemical densities in the CARES simulations
chem2rho_cares = {'SO4': 1770.0, 'NO3': 1770.0, 'Cl': 1900.0, 'NH4': 1770.0, 'MSA': 1800.0, 
    'ARO1': 1400.0, 'ARO2': 1400.0, 'ALK1': 1400.0, 'OLE1': 1400.0, 'API1': 1000.0, 'API2': 1400.0, 
    'LIM1': 1400.0, 'LIM2': 1400.0, 'CO3': 2600.0, 'Na': 1900.0, 'Ca': 2600.0, 'OIN': 2600.0, 
    'OC': 1000.0, 'BC': 1700.0, 'H2O': 1000.0}

# The chemical kappa values in the CARES simulations
chem2kappa_cares = {'SO4': 0.65, 'NO3': 0.65, 'Cl': 1.28, 'NH4': 0.65, 'MSA': 0.53, 
    'ARO1': 0.1, 'ARO2': 0.1, 'ALK1': 0.1, 'OLE1': 0.1, 'API1': 0.1, 'API2': 0.1, 
    'LIM1': 0.1, 'LIM2': 0.1, 'CO3': 0.53, 'Na': 1.28, 'Ca': 0.53, 'OIN': 0.1, 
    'OC': 0.001, 'BC': 0.0, 'H2O': 0.0}

# The chemical refraction indices in the CARES simulations
chem2refr_cares1 = {'SO4': (1.5+0j), 'NO3': (1.5+0j), 'Cl': (1.5+0j), 'NH4': (1.5+0j), 'MSA': (1.43+0j), 
    'ARO1': (1.45+0j), 'ARO2': (1.45+0j), 'ALK1': (1.45+0j), 'OLE1': (1.45+0j), 'API1': (1.45+0j), 
    'API2': (1.45+0j), 'LIM1': (1.45+0j), 'LIM2': (1.45+0j), 'CO3': (1.5+0j), 'Na': (1.5+0j), 
    'Ca': (1.5+0j), 'OIN': (1.55+0.006j), 'OC': (1.45+0j), 'BC': (1.82+0.74j), 'H2O': None}
chem2refr_cares = {f'{wvid}/{chem}': chmrfr for wvid in wvid2len 
    for chem, chmrfr in chem2refr_cares1.items()}
chem2refr_cares.update(h2o_refrs)

In [ ]:
info_fam = {
    ############################ The BlueWaters Chi Sampling Simulation ###########################

    ##### Variant I: Full Chemicals and Diameter Range ##### 
    # The number of time-steps in the simulation
    '01_bwchi/n_t': 25,
    # The simulation grid size
    '01_bwchi/n_grid': (1000, 1, 1),
    # The input data's relative directory
    '01_bwchi/inp/data_tree': '01_bwchisamp',
    # The function for obtaining the input `nc_path` and `snridx_nc`
    '01_bwchi/inp/nc_pathfn': get_bwncpath,
    # The relative path of the output
    '01_bwchi/out/nc_tree': '02_masshist/02_bwchisamp',
    # The maximum number of netCDF open files at a time
    '01_bwchi/n_maxfp': 1,
    # The number of diameter histogram bins
    '01_bwchi/n_bins': 20,
    # The lower range of the diamter histogram
    '01_bwchi/diam_low': 1e-9,
    # The higher range of the diamter histogram
    '01_bwchi/diam_high': 1e-4,
    # The chemical species to keep
    '01_bwchi/chem_species': 'all',
    # The mapping of chemical species (str) to physical density values (float, kg/m^3)
    '01_bwchi/chem2rho': chem2rho_bwchi,
    # The mapping of chemical species (str) to physical kappa values (float) for CCN calculaitons
    '01_bwchi/chem2kappa': chem2kappa_bwchi,
    # The mapping of chemical species (str) to refractive index values (complex) for optical calculations
    '01_bwchi/chem2refr': chem2refr_bwchi,
    # The mapping of wave length id (str) to wave length
    '01_bwchi/wvid2len': wvid2len,

    # The output file subtitle
    '01_bwchi/subtitle': dedent('''
        This file contains the chemical mass histograms from the
        PartMC run on the Bluewaters with the chi sampling. All 
        chemical species and the full range of diameters were 
        included in this data.'''),

    ##### Variant II: Reduced Chemicals and Diameter Range ####
    # The number of time-steps in the simulation
    '02_bwchi/n_t': 25,
    # The simulation grid size
    '02_bwchi/n_grid': (1000, 1, 1),
    # The input data's relative directory
    '02_bwchi/inp/data_tree': '01_bwchisamp',
    # The function for obtaining the input `nc_path` and `snridx_nc`
    '02_bwchi/inp/nc_pathfn': get_bwncpath,
    # The relative path of the output
    '02_bwchi/out/nc_tree': '02_masshist/03_bwchisamp',
    # The maximum number of netCDF open files at a time
    '02_bwchi/n_maxfp': 1,
    # The number of diameter histogram bins
    '02_bwchi/n_bins': 19,
    # The lower range of the diamter histogram
    '02_bwchi/diam_low': (10 ** (-8.75)),
    # The higher range of the diamter histogram
    '02_bwchi/diam_high': 1e-4,
    # The chemical species to keep
    '02_bwchi/chem_species': 'SO4,NO3,Cl,NH4,ARO1,ARO2,ALK1,OLE1,API1,Na,OIN,OC,BC,MOC,H2O',
    # The mapping of chemical species (str) to physical density values (float, kg/m^3)
    '02_bwchi/chem2rho': chem2rho_bwchi,
    # The mapping of chemical species (str) to physical kappa values (float) for CCN calculaitons
    '02_bwchi/chem2kappa': chem2kappa_bwchi,
    # The mapping of chemical species (str) to refractive index values (complex) for optical calculations
    '02_bwchi/chem2refr': chem2refr_bwchi,
    # The mapping of wave length id (str) to wave length
    '02_bwchi/wvid2len': wvid2len,

    # The output file subtitle
    '02_bwchi/subtitle': dedent('''
        This file contains the chemical mass histograms from the
        PartMC run on the Bluewaters with the chi sampling. A reduced 
        subset of chemical species and range of diameters were 
        included in this data.'''),

    #################################### The CARES Simulation ####################################
    
    ##### Variant I: Full Chemicals and Diameter Range ##### 
    # The number of time-steps in the simulation
    '01_cares/n_t': 9,
    # The simulation grid size
    '01_cares/n_grid': (169, 159, 39),
    # The input data's relative directory
    '01_cares/inp/data_tree': '03_caressamp',
    # The function for obtaining the input `nc_path` and `snridx_nc`
    '01_cares/inp/nc_pathfn': get_caresncpath,
    # The relative path of the output
    '01_cares/out/nc_tree': '02_masshist/04_caressamp',
    # The maximum number of netCDF open files at a time
    '01_cares/n_maxfp': 10,
    # The number of diameter histogram bins
    '01_cares/n_bins': 20,
    # The lower range of the diamter histogram
    '01_cares/diam_low': 1e-9,
    # The higher range of the diamter histogram
    '01_cares/diam_high': 1e-4,
    # The chemical species to keep
    '01_cares/chem_species': 'all',
    # The mapping of chemical species (str) to physical density values (float, kg/m^3)
    '01_cares/chem2rho': chem2rho_cares,
    # The mapping of chemical species (str) to physical kappa values (float) for CCN calculaitons
    '01_cares/chem2kappa': chem2kappa_cares,
    # The mapping of chemical species (str) to refractive index values (complex) for optical calculations
    '01_cares/chem2refr': chem2refr_cares,
    # The mapping of wave length id (str) to wave length
    '01_cares/wvid2len': wvid2len,

    # The output file subtitle
    '01_cares/subtitle': dedent('''
        This file contains the chemical mass histograms from the
        PartMC run on the CARES dataset. All chemical species and 
        the full range of diameters were included in this data.'''),

    ##### Variant II: Reduced Chemicals and Diameter Range ####
    # The number of time-steps in the simulation
    '02_cares/n_t': 9,
    # The simulation grid size
    '02_cares/n_grid': (169, 159, 39),
    # The input data's relative directory
    '02_cares/inp/data_tree': '03_caressamp',
    # The function for obtaining the input `nc_path` and `snridx_nc`
    '02_cares/inp/nc_pathfn': get_caresncpath,
    # The relative path of the output
    '02_cares/out/nc_tree': '02_masshist/05_caressamp',
    # The maximum number of netCDF open files at a time
    '02_cares/n_maxfp': 10,
    # The number of diameter histogram bins
    '02_cares/n_bins': 14,
    # The lower range of the diamter histogram
    '02_cares/diam_low': (10 ** (-8.25)),
    # The higher range of the diamter histogram
    '02_cares/diam_high': (10 ** (-4.75)),
    # The chemical species to keep
    '02_cares/chem_species': 'SO4,Cl,NH4,API1,Na,OC,BC',
    # The mapping of chemical species (str) to physical density values (float, kg/m^3)
    '02_cares/chem2rho': chem2rho_cares,
    # The mapping of chemical species (str) to physical kappa values (float) for CCN calculaitons
    '02_cares/chem2kappa': chem2kappa_cares,
    # The mapping of chemical species (str) to refractive index values (complex) for optical calculations
    '02_cares/chem2refr': chem2refr_cares,
    # The mapping of wave length id (str) to wave length
    '02_cares/wvid2len': wvid2len,

    # The output file subtitle
    '02_cares/subtitle': dedent('''
        This file contains the chemical mass histograms from the
        PartMC run on the CARES dataset. A reduced subset of chemical 
        species and range of diameters were included in this data.'''),    
}

In [ ]:
fam_data, need_work = args_datafamily, False

if my_rank is not None:
    # The number of time-steps in the simulation
    n_t_all = info_fam[f'{fam_data}/n_t']

    # The simulation grid size
    n_grid = info_fam[f'{fam_data}/n_grid']

    # The input data's relative directory
    data_tree = info_fam[f'{fam_data}/inp/data_tree']

    # The function for obtaining the input `nc_path` and `snridx_nc`
    nc_pathfninp = info_fam[f'{fam_data}/inp/nc_pathfn']

    # The relative path of the output
    nc_treeout = info_fam[f'{fam_data}/out/nc_tree']

    # The maximum number of netCDF open files at a time
    n_maxfp = info_fam[f'{fam_data}/n_maxfp']

    # The number of diameter histogram bins
    n_bins = info_fam[f'{fam_data}/n_bins']

    # The lower end of the diamter histogram
    diam_low = info_fam[f'{fam_data}/diam_low']

    # The higher end of the diamter histogram
    diam_high = info_fam[f'{fam_data}/diam_high']

    # The chemical species to keep
    chem_species_keep = info_fam[f'{fam_data}/chem_species']

    # The mapping of chemical species (str) to physical density values (float, kg/m^3)
    chem2rho = info_fam[f'{fam_data}/chem2rho']

    # The mapping of chemical species (str) to physical kappa values (float) for CCN calculaitons
    chem2kappa = info_fam[f'{fam_data}/chem2kappa']

    # The mapping of chemical species (str) to refractive index values (complex) for optical calculations
    chem2refr = info_fam[f'{fam_data}/chem2refr']

    # The mapping of wave length id (str) to wave length
    wvid2len = info_fam[f'{fam_data}/wvid2len']

    # The output file subtitle
    subtitle = info_fam[f'{fam_data}/subtitle']

    out_tree = '/'.join(nc_treeout.split('/')[:-1])
    nc_pathout = f'{out_datadir}/{nc_treeout}_{my_rank:02d}.nc'

    need_work = not(args_belazy and exists(nc_pathout))

if (my_rank is not None) and need_work:
    # Adjusting for dry runs!
    if args_dryrun:
        n_grid = tuple(min(aa, bb) for aa, bb in zip(n_grid, (5, 3, 4)))

    # Creating the output directory early on, so that any issues will be raised before the computations.
    outdir = f'{out_datadir}/{out_tree}'
    if not exists(outdir):
        os.makedirs(f'{out_datalfsdir}/{out_tree}', exist_ok=True)
        os.symlink(f'{out_datalfsdir}/{out_tree}', f'{out_datadir}/{out_tree}')
        assert exists(outdir)
    
    start_time = time.perf_counter()
    
    # The logarithmic base
    diam_logbase = np.log(10)

    # The grid size values in each x-, y-, and z-direction
    n_grid_x, n_grid_y, n_grid_z = n_grid
    # Defining the subset of scenario and time indecis to loop over
    snr_idxs = list(range(n_grid_x * n_grid_y * n_grid_z))
    t_idxs = list(range(n_t_all))

    my_snridxsnp = np.array_split(snr_idxs, n_workers)[my_rank]
    my_snridxs = my_snridxsnp.tolist()
    my_snridxsset = set(my_snridxs)
    n_waveref = len(wvid2len)

    n_snr, n_mysnr, n_t = len(snr_idxs), len(my_snridxs), len(t_idxs)
    n_mysamp = n_mysnr * n_t

    assert len(my_snridxs) > 0, f'n_workers={n_workers} too big for n_snr={n_snr}'

    # The file pointer cache
    nc_path2fp = odict()
    assert n_maxfp > 0

    # Compiling the list of targets for my rank
    prod_itrtr = product(range(n_grid_x), range(n_grid_y), range(n_grid_z))
    my_targets = [(scnr_idx, i_x, i_y, i_z)
        for scnr_idx, (i_x, i_y, i_z) in enumerate(prod_itrtr)
        if scnr_idx in my_snridxsset]

    mytqdm = tqdm if (my_rank == 0) else lambda aa: aa
    chem_species_str, n_chem, keydims = None, None, None
    hist_datalst = defaultdict(list)
    for scnr_idx, i_x, i_y, i_z in mytqdm(my_targets):
        if not use_tqdm:
            print('.', end='', flush=True)
        for t_idx in t_idxs:
            nc_path, snridx_nc = nc_pathfninp(data_dir=data_dir, 
                data_tree=data_tree, i_grid=(i_x, i_y, i_z), n_grid=n_grid, 
                scnr_idx=scnr_idx, t_idx=t_idx)
            
            # Looking up the file pointer cache
            if nc_path in nc_path2fp:
                nc_fp = nc_path2fp[nc_path]
            else:
                # Making sure we have at most `n_maxfp-1` open file 
                # pointers before we open a new one.
                while len(nc_path2fp) >= n_maxfp:
                    ncpath_old = next(iter(nc_path2fp))
                    ncfp_old = nc_path2fp.pop(ncpath_old)
                    ncfp_old.close()
                    
                nc_fp = Dataset(nc_path, "r")
                nc_path2fp[nc_path] = nc_fp

            hist_data = get_histdata(nc_fp, nc_path, snridx_nc, n_bins, diam_low, 
                diam_high, diam_logbase, chem_species_keep, lib='numpy')

            chemspcsstrthis = hist_data.pop('chem_species_str')
            chem_species_str = chemspcsstrthis if chem_species_str is None else chem_species_str
            assert chem_species_str == chemspcsstrthis, dedent(f'''
                    {chem_species_str} != {chemspcsstrthis}''')

            if keydims is None:
                keydims = {key: val.shape for key, val in hist_data.items()}

            assert set(hist_data) == set(keydims), dedent(f''''
                New unusal keys are being introduced later:
                    usual keys: {keydims.keys()}
                    new keys: {hist_data.keys()}''')

            for key, val in hist_data.items():
                assert val.shape == keydims[key], dedent(f''''
                    {key} has a different shape than usual:
                        usual shape: {keydims[key]}
                        new shape: {val.shape}''')

                hist_datalst[key].append(val)
    
    chem_species_lst = chem_species_str.split(',')
    
    # The chemical density values in kg / m^3.
    rho_chmnp = np.array([chem2rho[chem] for chem in chem_species_lst], dtype=np.float64)
    
    # The chemical kappa values for CCN calculations
    kappa_chmnp = np.array([chem2kappa[chem] for chem in chem_species_lst], dtype=np.float64)
    
    # The chemical refraction index values for optical property calculations
    wave_idssrtd = [wvid for wvid, wvlen in sorted(wvid2len.items(), key=lambda pair: pair[1])]
    refr_wvchmrefnp = np.array([
        [chem2refr[f'{wvid}/{chem}'] for chem in chem_species_lst] 
        for wvid in wave_idssrtd]).astype(np.complex128)
    len_wvrefnp = np.array([wvid2len[wvid] for wvid in wave_idssrtd])
    
    # Closing any remaining open file pointers
    for ncpath_old in list(nc_path2fp):
        ncfp_old = nc_path2fp.pop(ncpath_old)
        ncfp_old.close()

    hist_data = dict()
    for key, val_lst in hist_datalst.items():
        myval1 = np.stack(val_lst, axis=0)
        assert myval1.shape == (n_mysnr * n_t, *keydims[key])
        myval2 = myval1.reshape(n_mysnr, n_t, *keydims[key])
        assert myval2.shape == (n_mysnr, n_t, *keydims[key])
        hist_data[key] = myval2
        
    if my_rank == 0:
        print(f'\nFinished collecting the data from disk in {time.perf_counter()-start_time:.2f}!')


In [ ]:
# Writing the histogram data to disk
if (my_rank is not None) and need_work:
    # Extracting the histogram data
    n_chem = hist_data['m_chmprthst'].shape[-2]
    
    n_prthst = hist_data.pop('n_prthst')
    assert n_prthst.shape == (n_mysnr, n_t, n_bins)
    assert n_prthst.dtype == np.int64, n_prthst.dtype
    m_prthst = hist_data.pop('m_prthst')
    assert m_prthst.shape == (n_mysnr, n_t, n_bins)
    assert m_prthst.dtype == np.float64, m_prthst.dtype
    m_chmprthst = hist_data.pop('m_chmprthst')
    assert m_chmprthst.shape == (n_mysnr, n_t, n_chem, n_bins), dedent(f'''
        {m_chmprthst.shape} != {(n_mysnr, n_t, n_chem, n_bins)}''')
    assert m_chmprthst.dtype == np.float64, m_chmprthst.dtype

    # Making sure no data item is left over
    assert len(hist_data) == 0, f'{hist_data}'

    ncfile = Dataset(nc_pathout, mode='w', format='NETCDF4')
    # The number of entire scenarios
    nsnr_dim = ncfile.createDimension('n_snr', n_mysnr)
    # The number of entire scenarios
    nsnr_dim = ncfile.createDimension('n_snr_all', n_snr)
    # The number of time steps in each scenario
    nt_dim = ncfile.createDimension('n_t', n_t)
    # The number of entire samples
    nsamp_dim = ncfile.createDimension('n_samp', n_mysamp)
    # The number of diameter histogram bins
    kbins_dim = ncfile.createDimension('n_bins', n_bins)
    # The number of chemical specices
    nchem_dim = ncfile.createDimension('n_chem', n_chem)
    # The number of chemical specices
    nwave_dim = ncfile.createDimension('n_wave', n_waveref)
    # The x-axis grid size of the simulation
    ngridx_dim = ncfile.createDimension('n_grid_x', n_grid_x)
    # The y-axis grid size of the simulation
    ngridy_dim = ncfile.createDimension('n_grid_y', n_grid_y)
    # The z-axis grid size of the simulation
    ngridz_dim = ncfile.createDimension('n_grid_z', n_grid_z)
    # unit dimension
    one_dim = ncfile.createDimension('one', 1)

    # Keeping a record of which dimensions belonged to my rank
    i_targs = np.array(my_targets).astype(np.int64)
    assert i_targs.shape == (n_mysnr, 4)
    
    i_snr, i_x, i_y, i_z = i_targs.T
    assert i_snr.shape == (n_mysnr,)
    assert i_x.shape == (n_mysnr,)
    assert i_y.shape == (n_mysnr,)
    assert i_z.shape == (n_mysnr,)

    # Adding the file title/subtitle
    ncfile.title = 'The chemical mass histograms'
    ncfile.subtitle = subtitle

    # The range of the diamter histogram
    dlow_nc = ncfile.createVariable('diam_low', np.float64, ('one',))
    dlow_nc.units = 'm'
    dlow_nc.long_name = 'diameter_lower_bound'

    dhigh_nc = ncfile.createVariable('diam_high', np.float64, ('one',))
    dhigh_nc.units = 'm'
    dhigh_nc.long_name = 'diameter_upper_bound'

    # The logarithmic base
    logbase_nc = ncfile.createVariable('diam_logbase', np.float64, ('one',))
    logbase_nc.units = 'none'
    logbase_nc.long_name = 'diameter_log_base'

    # The chemical species names
    chmspcs_nc = ncfile.createVariable('chem_species', np.int64, ('n_chem',))
    chmspcs_nc.units = 'none'
    chmspcs_nc.names = chem_species_str
    chmspcs_nc.long_name = chem_species_str

    # The chemical density values in kg / m^3.
    chmrho_nc = ncfile.createVariable('chem_rho', np.float64, ('n_chem',))
    chmrho_nc.units = 'kg/m^3'
    chmrho_nc.long_name = 'chem_density'

    # The chemical kappa values for CCN calculations
    chmkappa_nc = ncfile.createVariable('chem_kappa', np.float64, ('n_chem',))
    chmkappa_nc.units = 'none'
    chmkappa_nc.long_name = 'chem_kappa'

    # The wave lengths for optical property calculations
    lenwv_nc = ncfile.createVariable('wave_len', np.float64, ('n_wave',))
    lenwv_nc.units = 'none'
    lenwv_nc.long_name = 'wave_len'

    # The real chemical refraction index values for optical property calculations
    chmrefrreal_nc = ncfile.createVariable('chem_refr_real', np.float64, ('n_wave', 'n_chem'))
    chmrefrreal_nc.units = 'none'
    chmrefrreal_nc.long_name = 'chem_refraction_real'

    # The imaginary chemical refraction index values for optical property calculations
    chmrefrimag_nc = ncfile.createVariable('chem_refr_imag', np.float64, ('n_wave', 'n_chem'))
    chmrefrimag_nc.units = 'none'
    chmrefrimag_nc.long_name = 'chem_refraction_imag'

    # Writing the data to the netCDF file
    n_prthst_nc = ncfile.createVariable('n_prthst', np.int64, ('n_samp', 'n_bins'))
    n_prthst_nc.units = 'prt/log(m)'
    n_prthst_nc.long_name = 'num_particles_histogram'

    m_prthst_nc = ncfile.createVariable('m_prthst', np.float64, ('n_samp', 'n_bins'))
    m_prthst_nc.units = 'kg/log(m)'
    m_prthst_nc.long_name = 'mass_particles_histogram_net'

    m_chmprthst_nc = ncfile.createVariable('m_chmprthst', np.float64, ('n_samp', 'n_chem', 'n_bins'))
    m_chmprthst_nc.units = 'kg/log(m)'
    m_chmprthst_nc.long_name = 'mass_particles_histogram_speciated'

    i_snr_nc = ncfile.createVariable('i_snr', np.int64, ('n_snr',))
    i_snr_nc.units = 'one'
    i_snr_nc.long_name = 'included_snr_idxs_in_my_rank'

    i_x_nc = ncfile.createVariable('i_x', np.int64, ('n_snr',))
    i_x_nc.units = 'one'
    i_x_nc.long_name = 'included_x_grid_idxs_in_my_rank'

    i_y_nc = ncfile.createVariable('i_y', np.int64, ('n_snr',))
    i_y_nc.units = 'one'
    i_y_nc.long_name = 'included_y_grid_idxs_in_my_rank'

    i_z_nc = ncfile.createVariable('i_z', np.int64, ('n_snr',))
    i_z_nc.units = 'one'
    i_z_nc.long_name = 'included_z_grid_idxs_in_my_rank'

    dlow_nc[:], dhigh_nc[:], logbase_nc[:] = diam_low, diam_high, diam_logbase
    chmspcs_nc[:] = np.arange(n_chem)
    n_prthst_nc[:] = n_prthst.reshape(n_mysamp, n_bins)
    m_prthst_nc[:] = m_prthst.reshape(n_mysamp, n_bins)
    m_chmprthst_nc[:] = m_chmprthst.reshape(n_mysamp, n_chem, n_bins)
    i_snr_nc[:], i_x_nc[:], i_y_nc[:], i_z_nc[:] = i_snr, i_x, i_y, i_z
    chmrho_nc[:], chmkappa_nc[:] = rho_chmnp, kappa_chmnp
    chmrefrreal_nc[:], chmrefrimag_nc[:] = refr_wvchmrefnp.real, refr_wvchmrefnp.imag
    lenwv_nc[:] = len_wvrefnp

    # Closing the netCDF file
    ncfile.close()
    if my_rank == 0:
        print('Done!', flush=True)

# Compiling All Data into a Single netCDF File

In [ ]:
fam_data = None

# NetCDF Dimension Compilation Rules
dim_cmplrules = {'n_snr': 'sum', 'n_t': 'same', 'n_samp': 'sum', 'n_snr_all': 'same',
    'n_bins': 'same', 'n_chem': 'same', 'n_wave': 'same', 'n_grid_x': 'same', 
    'n_grid_y': 'same', 'n_grid_z': 'same', 'one': 'same'}

# NetCDF Variable Compilation Rules
var_cmplrules = {
    'diam_low': ('same', np.float64), 
    'diam_high': ('same', np.float64), 
    'diam_logbase': ('same', np.float64), 
    'chem_species': ('same', np.int64), 
    'chem_rho': ('same', np.float64), 
    'chem_kappa': ('same', np.float64), 
    'chem_refr_real': ('same', np.float64), 
    'chem_refr_imag': ('same', np.float64), 
    'wave_len': ('same', np.float64), 
    'n_prthst': ('cat', np.int64), 
    'm_prthst': ('cat', np.float64), 
    'm_chmprthst': ('cat', np.float64), 
    'i_snr': ('cat', np.int64), 
    'i_x': ('cat', np.int64), 
    'i_y': ('cat', np.int64), 
    'i_z': ('cat', np.int64)}

if fam_data is not None:
    ################################ Phase I #################################
    ############## Reading all the data from all the rank files ##############

    # The relative path of the output
    nc_treeout = info_fam[f'{fam_data}/out/nc_tree']

    rank2ncfp = dict()
    out_tree = '/'.join(nc_treeout.split('/')[:-1])

    rank_ncpaths = glob.glob(f'{out_datadir}/{nc_treeout}_*.nc')
    rank_strs_ = [ncpth.split('_')[-1].split('.')[0] for ncpth in rank_ncpaths]
    ranks_lst = [int(rnkstr) for rnkstr in rank_strs_ if rnkstr.isdigit()]
    ranks_lst = sorted(ranks_lst)
    assert ranks_lst == list(range(len(ranks_lst)))

    for rank in ranks_lst:
        nc_pathrank = f'{out_datadir}/{nc_treeout}_{rank:02d}.nc'
        nc_fp = Dataset(nc_pathrank, mode='r', format='NETCDF4')
        rank2ncfp[rank] = nc_fp

    # Compiling NetCDF Dimensions
    dims_cmpld = dict()
    for rank, nc_fp in rank2ncfp.items():
        # all keys must be accounted for
        rank_dims = {key: val.size for key, val in nc_fp.dimensions.items()}
        assert set(rank_dims) == set(dim_cmplrules), dedent(f'''
            {rank_dims.keys()} != {dim_cmplrules.keys()}''')
        
        # summing or same-proofing each dimensions
        for key, cmpl_rule in dim_cmplrules.items():
            dim = rank_dims[key]
            if cmpl_rule == 'sum':
                dims_cmpld.setdefault(key, 0)
                dims_cmpld[key] += dim
            elif cmpl_rule == 'same':
                dims_cmpld.setdefault(key, dim)
                assert dims_cmpld[key] == dim
            else:
                raise ValueError(f'undefined cmpl_rule = {cmpl_rule}')

    # Compiling NetCDF Attributes
    attrs_cmpld = dict(vars(rank2ncfp[ranks_lst[0]]))
    assert all(dict(vars(nc_fp)) == attrs_cmpld 
        for rank, nc_fp in rank2ncfp.items())

    # Compiling NetCDF Variables
    vars_catlst = defaultdict(list)
    vars_cmpld = dict()
    varattrs_cmpld = dict()
    vardimnames_cmpld = dict()

    for rank, nc_fp in rank2ncfp.items():
        # Making sure the variables all have defined rules
        assert set(nc_fp.variables) == set(var_cmplrules), dedent(f'''
            {nc_fp.variables.keys()} != {var_cmplrules.keys()}''')
            
        for key, (cmpl_rule, key_dtype) in var_cmplrules.items():
            rank_varbl = nc_fp.variables[key]

            # Making sure the key has the anticipated dtype
            assert rank_varbl.dtype == key_dtype, f'{rank_varbl.dtype} != {key_dtype}'

            # appending or same-proofing each value
            if cmpl_rule == 'same':
                if key not in vars_cmpld:
                    vars_cmpld[key] = rank_varbl[:]
                assert np.all(vars_cmpld[key] == rank_varbl)
            elif cmpl_rule == 'cat':
                vars_catlst[key].append(rank_varbl[:])
            else:
                raise ValueError(f'undefined cmpl_rule = {cmpl_rule}')

            # The variable attributes should be identical
            if key not in varattrs_cmpld:
                varattrs_cmpld[key] = dict(vars(rank_varbl))
            assert varattrs_cmpld[key] == dict(vars(rank_varbl))

            # The variable dimension names should be identical
            if key not in vardimnames_cmpld:
                vardimnames_cmpld[key] = rank_varbl.dimensions
            assert vardimnames_cmpld[key] == rank_varbl.dimensions

    # Concatenating all the values
    for key, arr_list in vars_catlst.items():
        vars_cmpld[key] = np.concatenate(arr_list, axis=0)

    # Closing all the open file pointers
    for rank, nc_fp in rank2ncfp.items():
        nc_fp.close()

    ################################ Phase II ################################
    ######################## Writing the Compiled Data #######################

    nc_pathout = f'{out_datadir}/{nc_treeout}.nc'
    nc_fpout = Dataset(nc_pathout, mode='w', format='NETCDF4')

    # Writing the dimensions
    for dim_name, dim_val in dims_cmpld.items():
        nc_fpout.createDimension(dim_name, dim_val)

    # Writing the attributes
    for attr_name, attr_val in attrs_cmpld.items():
        setattr(nc_fpout, attr_name, attr_val)

    # Writing the variables
    for key, varbl in vars_cmpld.items():
        key_cmplrule, key_dtype = var_cmplrules[key]
        key_dimnames = vardimnames_cmpld[key]
        nc_var = nc_fpout.createVariable(key, key_dtype, key_dimnames)
        for attr_name, attr_val in varattrs_cmpld[key].items():
            setattr(nc_var, attr_name, attr_val)
        nc_var[:] = varbl

    # Closing the netCDF file
    nc_fpout.close()


# Loading the Histogram Data

In [ ]:
fam_data, grid_ssfrqs = '02_bwchi', None
fam_data, grid_ssfrqs = '02_cares', (10, 10, 10, 1)

nc_treeout = info_fam[f'{fam_data}/out/nc_tree']
data_path = f'{data_dir}/{nc_treeout}.nc'
assert data_path.endswith('.nc')
n_grid_x, n_grid_y, n_grid_z = info_fam[f'{fam_data}/n_grid']

with Dataset(data_path, "r") as fp:
    ################# The Array Dimensions #################
    # The number of scenarios
    n_snrall = fp.dimensions['n_snr'].size
    # The number of time steps in each scenario
    n_t = fp.dimensions['n_t'].size
    # The number of diameter histogram bins
    n_bins = fp.dimensions['n_bins'].size
    # The number of chemical specices
    n_chem = fp.dimensions['n_chem'].size
    # The number of wave lengths
    n_wave = fp.dimensions['n_wave'].size

    # Applying the sub-sampling
    if grid_ssfrqs is not None:
        x_ssfrq, y_ssfrq, z_ssfrq, t_ssfrq = grid_ssfrqs
        assert n_snrall == (n_grid_x * n_grid_y * n_grid_z)
        i_snrs1 = np.arange(n_snrall * n_t).reshape(n_grid_x, n_grid_y, n_grid_z, n_t)
        assert i_snrs1.shape == (n_grid_x, n_grid_y, n_grid_z, n_t)
        i_snrs2 = i_snrs1[::x_ssfrq, ::y_ssfrq, ::z_ssfrq, ::t_ssfrq]
        n_samp = i_snrs2.size
        n_snr, n_t = math.prod(i_snrs2.shape[:-1]), i_snrs2.shape[-1]
        i_snrs = i_snrs2.ravel()
        assert i_snrs.shape == (n_samp,)
    else:
        n_snr, n_samp = n_snrall, n_snrall * n_t
        i_snrs = slice(None, None, None)

    #################### The Data Arrays ####################
    # The number of particles within each diameter bin
    n_prthst_ = fp.variables['n_prthst'][i_snrs]
    assert n_prthst_.shape == (n_samp, n_bins)
    n_prthst = n_prthst_.reshape(n_snr, n_t, n_bins)
    assert n_prthst.shape == (n_snr, n_t, n_bins)

    # The mass within each diameter bin (the mass histogram)
    m_prthst_ = fp.variables['m_prthst'][i_snrs]
    assert m_prthst_.shape == (n_samp, n_bins)
    m_prthst = m_prthst_.reshape(n_snr, n_t, n_bins)
    assert m_prthst.shape == (n_snr, n_t, n_bins)

    # The mass of each chemical species within the diameter bins
    m_chmprthst_ = fp.variables['m_chmprthst'][i_snrs]
    assert m_chmprthst_.shape == (n_samp, n_chem, n_bins)
    m_chmprthst = m_chmprthst_.reshape(n_snr, n_t, n_chem, n_bins)
    assert m_chmprthst.shape == (n_snr, n_t, n_chem, n_bins)

    ####################### Meta-Data #######################
    # The Chemical Species
    chem_species_str = fp.variables['chem_species'].names

    # The chemical density for each species in (kg/m^3)
    rho_chm = fp.variables['chem_rho'][:]
    assert rho_chm.shape == (n_chem,)
    # The kappa of the chemical species.
    kappa_chm = fp.variables['chem_kappa'][:]
    assert kappa_chm.shape == (n_chem,)
    # The real refraction indices of the chemical species at the 550nm wave length.
    refr_wvchem_real = fp.variables['chem_refr_real'][:]
    assert refr_wvchem_real.shape == (n_wave, n_chem)
    # The imaginary refraction indices of the chemical species at the 550nm wave length.
    refr_wvchem_imag = fp.variables['chem_refr_imag'][:]
    assert refr_wvchem_imag.shape == (n_wave, n_chem)
    # The complex refraction indices of the chemical species at the 550nm wave length.
    refr_wvchem = refr_wvchem_real + 1j * refr_wvchem_imag
    assert refr_wvchem.shape == (n_wave, n_chem)

    # The histogram bin lower range of diameters
    diam_low = fp.variables['diam_low'][:].item()
    # The histogram bin higher range of diameters
    diam_high = fp.variables['diam_high'][:].item()
    # The diameter log base
    diam_logbase = fp.variables['diam_logbase'][:].item()

    logdiam_low, logdiam_high = np.log(diam_low) / diam_logbase, np.log(diam_high) / diam_logbase
    logdiams = np.linspace(logdiam_low, logdiam_high, n_bins + 1, endpoint=True)
    x_histbins = np.exp(logdiams * diam_logbase)
    assert x_histbins.shape == (n_bins + 1,)

# Plotting the Mass Stacked Histogram Data

In [ ]:
plt.style.use('dark_background')

nrows, ncols = {'02_bwchi': (5, 5), '01_cares': (3, 3), '02_cares': (3, 3)}[fam_data]
fig, axes = plt.subplots(nrows, ncols, figsize=(3.2*ncols, 2.8*nrows), sharex=True, sharey=True)
axes = np.array(axes).reshape(nrows * ncols)

# The index of the scenario to show
snr_idx = 0
# Whether to normalize the values by the diameter bin widhts
do_normalize = True
# The y-axis mass unit (either 'kg' or 'g')
mass_unit = 'g'

logdiam_low, logdiam_high = np.log(diam_low) / diam_logbase, np.log(diam_high)/diam_logbase
for t_idx in range(nrows * ncols):
    m_prthst_t = m_prthst[snr_idx, t_idx]
    assert m_prthst_t.shape == (n_bins,)
    m_chmprthst_t = m_chmprthst[snr_idx, t_idx].T
    assert m_chmprthst_t.shape == (n_bins, n_chem)
    
    if do_normalize:
        bin_width = (logdiam_high - logdiam_low) / n_bins
        m_prthst_t = m_prthst_t / bin_width
        m_chmprthst_t = m_chmprthst_t / bin_width

    m_mul = {'g': 1000, 'kg': 1}[mass_unit]
    m_prthst_t = m_prthst_t * m_mul / bin_width
    m_chmprthst_t = m_chmprthst_t * m_mul / bin_width

    m_chmcs = np.cumsum(m_chmprthst_t, axis=1)
    assert m_chmcs.shape == (n_bins, n_chem)

    logdiams = np.linspace(logdiam_low, logdiam_high, n_bins, endpoint=False)
    bin_width = (logdiam_high - logdiam_low) / n_bins
    y_prfxstr = ('(Norm.) ' if do_normalize else '')

    ax = axes[t_idx]
    i_row, i_col = t_idx // ncols, t_idx % ncols
    colors = sns.color_palette('bright', n_chem)
    for i_chm in range(n_chem): 
        ax.bar(logdiams, m_chmprthst_t[:, i_chm], width=bin_width, align='edge',
            edgecolor='black', bottom=m_chmcs[:, i_chm-1] if i_chm > 0 else 0, color=colors[i_chm])
    ax.set_xlim(logdiam_low, logdiam_high)

    ax.set_title(f'Time={t_idx:02d}:00:00')
    if i_row == (nrows - 1):
        ax.set_xlabel('Diameter')
    if i_col == 0:
        ax.set_ylabel(f'{y_prfxstr}Species Mass')

    engfmtx = EngFormatter(unit='m', sep='')
    x_ticks = ax.get_xticks()
    x_ticklabels = [engfmtx(np.exp(xtck * diam_logbase)) for xtck in x_ticks]
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_ticklabels)

    ax.set_yscale('symlog', linthresh=1e-11)
    engfmty = EngFormatter(sep='', unit=mass_unit)
    ax.yaxis.set_major_formatter(engfmty)

fig.set_tight_layout({"w_pad": 0.0})

# Plotting the Mass Pre-Processing Statistic

In [ ]:
import warnings
import matplotlib
from copy import deepcopy
from mpl_toolkits.axes_grid1 import make_axes_locatable
warnings.filterwarnings("ignore", 
    message="divide by zero encountered in log10")

def pp_data(pp_id, eps, exponent):
    m_chmprthst1 = np.array(m_chmprthst) + eps
    assert m_chmprthst1.shape == (n_snr, n_t, n_chem, n_bins)

    if pp_id == 'legacy':
        m_chmprthst1b = m_chmprthst1
        assert m_chmprthst1b.shape == (n_snr, n_t, n_chem, n_bins)
    elif pp_id == 'root':
        m_chmprthst1b = np.power(m_chmprthst1, exponent)
        assert m_chmprthst1b.shape == (n_snr, n_t, n_chem, n_bins)
    else:
        raise ValueError(f'undefined pp_id = {pp_id}')

    m_chmprthst2 = m_chmprthst1b.reshape(n_snr * n_t, n_chem * n_bins)
    assert m_chmprthst2.shape == (n_snr * n_t, n_chem * n_bins)

    m_chmprthst3 = (m_chmprthst2**2).sum(axis=-1, keepdims=True) ** 0.5
    assert m_chmprthst3.shape == (n_snr * n_t, 1)

    m_chmprthst4 = m_chmprthst2 / m_chmprthst3
    assert m_chmprthst4.shape == (n_snr * n_t, n_chem * n_bins)

    m_chmprthst5 = m_chmprthst4.mean(axis=0)
    assert m_chmprthst5.shape == (n_chem * n_bins,)

    m_chmprthst6 = m_chmprthst4.std(axis=0)
    assert m_chmprthst6.shape == (n_chem * n_bins,)

    if pp_id == 'legacy':
        m_chmprthst7 = np.log(m_chmprthst3)
        assert m_chmprthst7.shape == (n_snr * n_t, 1)
    elif pp_id == 'root':
        m_chmprthst7 = np.power(m_chmprthst3, exponent)
        assert m_chmprthst7.shape == (n_snr * n_t, 1)
    else:
        raise ValueError(f'undefined pp_id = {pp_id}')

    m_chmprthst8 = m_chmprthst7.reshape(n_snr * n_t, 1, 1)
    assert m_chmprthst8.shape == (n_snr * n_t, 1, 1)

    m_chmprthst9 = m_chmprthst8.mean(axis=0, keepdims=True)[None]
    assert m_chmprthst9.shape == (1, 1, 1, 1)

    m_chmprthst10 = m_chmprthst8.std(axis=0, keepdims=True)[None]
    assert m_chmprthst10.shape == (1, 1, 1, 1)


    m_chmprthst11 = m_chmprthst4.reshape(n_snr * n_t, n_chem, n_bins)
    assert m_chmprthst11.shape == (n_snr * n_t, n_chem, n_bins)
    
    if pp_id == 'legacy':
        m_chmprthst12 = m_chmprthst11.mean(axis=0, keepdims=True)
        assert m_chmprthst12.shape == (1, n_chem, n_bins)

        m_chmprthst13 = m_chmprthst11.std(axis=0, keepdims=True)
        assert m_chmprthst13.shape == (1, n_chem, n_bins)
    elif pp_id == 'root':
        m_chmprthst12 = m_chmprthst11.mean(axis=(0, -1), keepdims=True)
        assert m_chmprthst12.shape == (1, n_chem, 1)
        
        m_chmprthst13 = m_chmprthst11.std(axis=(0, -1), keepdims=True)
        assert m_chmprthst13.shape == (1, n_chem, 1)
    else:
        raise ValueError(f'undefined pp_id = {pp_id}')
    
    m_chmprthst14 = (m_chmprthst11 - m_chmprthst12) / m_chmprthst13
    assert m_chmprthst14.shape == (n_snr * n_t, n_chem, n_bins)
    
    m_chmprthst15 = ((m_chmprthst14**2).mean(axis=0)**0.5).ravel()
    assert m_chmprthst15.shape == (n_chem * n_bins,)
    
    globals().update(**locals())

def plot_heatmap(plt_data, x_histbins):
    fig, ax = plt.subplots(1, 1, dpi=100, figsize=(6, 6))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.5)

    cmap = matplotlib.cm.Reds
    cmap.set_bad('white',1.)
    im = ax.imshow(plt_data, cmap=cmap)

    xticks = np.arange(n_bins + 1)
    assert len(xticks) == len(x_histbins)
    ax.set_xticks(xticks-0.501, minor=True)
    ax.set_xticks(xticks[::4]-0.5)
    ax.set_xticklabels(['$10^{' + f'{x/4-9}' + '}$' for x in xticks[::4]])

    x_ticks, x_ticklabels = [], []
    for i_tick, x_tick in enumerate(x_histbins):
        xtlog1 = np.log10(x_tick)
        if np.abs(xtlog1 - np.round(xtlog1)) < 0.001:
            x_ticks.append(i_tick - 0.5)
            x_ticklabels.append('$10^{' + f'{int(np.round(xtlog1))}' + '}$')
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_ticklabels)

    yticks = np.arange(n_chem)
    ax.set_yticks(yticks)
    ax.set_yticks(yticks-0.5, minor=True)
    ax.set_yticklabels(chem_species_str.split(','))

    ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

    # Remove minor ticks
    ax.tick_params(which='minor', bottom=False, left=False)

    fig.colorbar(im, cax=cax, orientation='vertical')

    ax.set_xlabel('Diameter (m)')
    ax.set_ylabel('Chemical Species')
    return fig, ax


# Plain Data without Pre-Processing

In [ ]:
m_chmprthst20 = m_chmprthst.reshape(n_snr * n_t, n_chem, n_bins)

In [ ]:
plt_data = m_chmprthst20.mean(axis=0).reshape(n_chem, n_bins)
fig, ax = plot_heatmap(plt_data, x_histbins)
ax.set_title('Plain Mean');

In [ ]:
plt_data = m_chmprthst20.std(axis=0).reshape(n_chem, n_bins)
fig, ax = plot_heatmap(plt_data, x_histbins)
ax.set_title('Plain Standard Deviation');

In [ ]:
plt_data = np.log10(m_chmprthst20.std(axis=0)).reshape(n_chem, n_bins)
fig, ax = plot_heatmap(plt_data, x_histbins)
ax.set_title('Log Standard Deviation');

## Legacy Pre-Processing

In [ ]:
pp_data(pp_id='legacy', eps=1e-25, exponent=0.5)

In [ ]:
plt.style.use('dark_background')

fig, ax = plt.subplots(1, 1, dpi=140, figsize=(5, 4))
plt_data = np.log10(np.abs(m_chmprthst6))
ax.hist(plt_data.ravel(), bins=17, range=[-17, 0], color='green', edgecolor='white', linewidth=2)
xticks = [-16, -14, -12, -10,  -8,  -6,  -4,  -2,   0]
ax.set_xticks(xticks)
# xticks = ax.get_xticks()
ax.set_xticklabels([r'$10^{'+ str(int(xx)) + r'}$' for xx in xticks])
ax.set_xlabel(r'$std[\tilde{x}]$')

x_vline = np.quantile(plt_data, .622)
ax.axvline(x=x_vline, ymin=0, ymax=0.98, lw=2, color='white', ls='--')
ax.text(x_vline*0.9, 120, '     60th\nPercentile',
        bbox={'facecolor': 'black', 'pad': 2})
ax.set_ylabel('Count');

### The Unscaled Overall Variation Heatmap

In [ ]:
plt_data = m_chmprthst6.reshape(n_chem, n_bins)
fig, ax = plot_heatmap(plt_data, x_histbins)

### The Unscaled Overall Variation Heatmap (Log-scaled colors)

In [ ]:
plt_data = np.log10(m_chmprthst6.reshape(n_chem, n_bins))
fig, ax = plot_heatmap(plt_data, x_histbins)

### Specific Scenario and Time-step Heatmap

In [ ]:
plt_data = np.log10(m_chmprthst[2, 24])
fig, ax = plot_heatmap(plt_data, x_histbins)

## Root Pre-Processing

### The Final RMSE in each Dimension

In [ ]:
pp_data(pp_id='root', eps=1e-225, exponent=1)
plt_data = deepcopy(m_chmprthst15).reshape(n_chem, n_bins)
# plt_data[[4, 10, 11, 12, 13, 15], :] = 0
# plt_data[:, 0] = 0
fig, ax = plot_heatmap(plt_data, x_histbins)